In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'


In [3]:
df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)
df_houses_pool = pd.read_csv(pathdata + 'data_output/Base_pool_hogares_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)

In [4]:
## Defino funciones dimension mercado de trabajo e ingresos

# Componente: Inserción laboral

# T1. 
# Variable: Actividad productiva
# Indicador: Ratio M/V en tasa de empleo

def actividad_productiva(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):
    
    """
    Calcula la actividad productiva, a partir de la proporción de tasas de empleo entre hombres y mujeres, junto con los errores asociados.

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tasa (DataFrame): DataFrame con las tasas de empleo (M/V) desglosadas por Aglomerado o Provincia. Expresado en % [0-100]
        size (DataFrame): DataFrame con los numeros de observaciones utilizados para calcular tasas de empleo (M/V). Expresado en valores absolutos
        ratio (DataFrame): DataFrame con las proporciones de tasas de empleo (M/V). Expresado en % [0-100]
        error (DataFrame): DataFrame con los errores asociados a las proporciones. Expresado en % [0-100]. 
                            LI: Limite inferior, LS: Limite superior, ME: Margen de error, ER: Error relativo (CV)


    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    numerador = df_temp[df_temp['ESTADO'] == 1].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)
    denominador = df_temp.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_temp[df_temp['ESTADO'] == 1].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Actividad productiva'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [5]:
tasa, size, ratio, error = actividad_productiva(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)

empleo = pd.concat([tasa, size, ratio, error], axis=1)

empleo

,Varon,Mujer,N_v,N_m,Actividad productiva,LI,LS,ME,ER
PROVINCIA,,,,,,,,,
2,78.710304,70.957452,642,594,90.150144,85.317535,94.982752,4.832609,5.360622
6,75.153672,55.778992,3985,3250,74.219915,72.048389,76.391442,2.171526,2.925800
10,69.529090,52.433530,1026,813,75.412363,70.971985,79.852740,4.440377,5.888129
14,79.622598,54.541412,1906,1487,68.499915,65.389416,71.610414,3.110499,4.540880
18,72.778626,48.508609,924,661,66.652274,61.882320,71.422227,4.769953,7.156475
22,73.566052,44.943799,829,608,61.093124,56.128739,66.057509,4.964385,8.125930
26,73.485409,55.669633,1176,973,75.756036,71.742573,79.769499,4.013463,5.297879
30,73.334174,51.223948,1496,1100,69.850038,66.142374,73.557702,3.707664,5.308034
34,64.272893,32.303294,841,459,50.259593,44.893889,55.625296,5.365703,10.675979


In [6]:
# Componente: Inserción laboral

# T2. 
# Variable: Busqueda de trabajo
# Indicador: Ratio M/V en tasa de desempleo

def busqueda_trabajo(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):
    
    """
    Calcula la actividad productiva, a partir de la proporción de tasas de desempleo entre hombres y mujeres, junto con los errores asociados.

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tasa (DataFrame): DataFrame con las tasas de desempleo (M/V) desglosadas por Aglomerado o Provincia. Expresado en % [0-100]
        size (DataFrame): DataFrame con los numeros de observaciones utilizados para calcular tasas de desempleo (M/V). Expresado en valores absolutos
        ratio (DataFrame): DataFrame con las proporciones de tasas de desempleo (M/V). Expresado en % [0-100]
        error (DataFrame): DataFrame con los errores asociados a las proporciones. Expresado en % [0-100]. 
                            LI: Limite inferior, LS: Limite superior, ME: Margen de error, ER: Error relativo (CV)


    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    numerador = df_temp[df_temp['ESTADO'] == 2].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)
    df_estado = df_temp[(df_temp['ESTADO'] == 1) | (df_temp['ESTADO'] == 2)]
    denominador = df_estado.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_temp[df_temp['ESTADO'] == 2].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Varon']
    p2 = tasa['Mujer']
    n1 = size['N_v']
    n2 = size['N_m']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Búsqueda de trabajo'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [7]:
tasa, size, ratio, error = busqueda_trabajo(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)

desempleo = pd.concat([tasa, size, ratio, error], axis=1)

desempleo

,Varon,Mujer,N_v,N_m,Búsqueda de trabajo,LI,LS,ME,ER
PROVINCIA,,,,,,,,,
2,6.270256,5.029093,39,36,124.679654,114.246343,135.112966,10.433311,8.368095
6,8.252182,10.440042,301,342,79.043576,74.553036,83.534116,4.490540,5.681094
10,4.901536,8.311059,55,59,58.976074,49.911190,68.040958,9.064884,15.370443
14,6.333984,12.016204,129,164,52.712020,46.198097,59.225943,6.513923,12.357567
18,4.921424,4.541953,43,27,108.354805,98.181874,118.527737,10.172931,9.388537
22,4.864433,4.154527,44,27,117.087538,107.235796,126.939280,9.851742,8.413997
26,4.906585,3.513886,71,43,139.634140,132.182078,147.086201,7.452062,5.336848
30,5.460783,10.521391,94,96,51.901716,44.235584,59.567849,7.666133,14.770480
34,5.816408,2.355315,50,13,246.948214,236.457868,257.438559,10.490346,4.247994


In [8]:
# Componente: Inserción laboral

# T3. 
# Variable: Acceso a derechos laborales
# Indicador: Ratio M/V en tasa de registro de la relación laboral

def acceso_derechos_laborales(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):
    
    """
    Calcula el acceso a derechos laborales, a partir de la proporción de tasa de registro de la relación laboral entre hombres y mujeres junto con los errores asociados.

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tasa (DataFrame): DataFrame con las tasas de registro (M/V) desglosadas por Aglomerado o Provincia. Expresado en % [0-100]
        size (DataFrame): DataFrame con los numeros de observaciones utilizados para calcular tasas de registro (M/V). Expresado en valores absolutos
        ratio (DataFrame): DataFrame con las proporciones de tasas de registro (M/V). Expresado en % [0-100]
        error (DataFrame): DataFrame con los errores asociados a las proporciones. Expresado en % [0-100]. 
                            LI: Limite inferior, LS: Limite superior, ME: Margen de error, ER: Error relativo (CV)


    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    df_estado = df_temp[(df_temp['ESTADO'] == 1) & (df_temp['CAT_OCUP'] == 3)]          # personas ocupadas asalariadas
    numerador = df_estado[df_estado['PP07H']==1].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    denominador = df_estado.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_temp[df_temp['ESTADO'] == 2].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Acceso a derechos laborales'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [9]:
tasa, size, ratio, error = acceso_derechos_laborales(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)

empleo_asalariado_con_descuento = pd.concat([tasa, size, ratio, error], axis=1)

empleo_asalariado_con_descuento

,Varon,Mujer,N_v,N_m,Acceso a derechos laborales,LI,LS,ME,ER
PROVINCIA,,,,,,,,,
2,79.665069,76.119022,39,36,95.548805,76.746181,114.351429,18.802624,19.678555
6,64.819435,61.219465,301,342,94.446157,86.978229,101.914085,7.467928,7.907075
10,61.784084,60.456728,55,59,97.851622,79.947232,115.756013,17.904391,18.297490
14,54.908631,50.804343,129,164,92.525240,81.024238,104.026242,11.501002,12.430123
18,55.114564,53.627512,43,27,97.301889,73.326441,121.277338,23.975449,24.640271
22,45.381320,56.913699,44,27,125.412171,101.636297,149.188046,23.775875,18.958188
26,79.404827,71.502111,71,43,90.047562,73.600134,106.494990,16.447428,18.265267
30,65.913125,56.866193,94,96,86.274460,72.491521,100.057399,13.782939,15.975688
34,50.218551,57.363368,50,13,114.227444,83.981953,144.472935,30.245491,26.478305


In [10]:
df_temp = df_people_pool.query('CH06 >= 16 & CH06 < 65')
df_estado = df_temp[(df_temp['ESTADO'] == 1) & (df_temp['CAT_OCUP'] == 3)]          # personas ocupadas asalariadas

# Filtrar y contar las combinaciones por género
resultados = df_estado.groupby(['CH04', 'PP07H', 'PP07I'])['PONDERA_new'].sum().unstack(level='CH04', fill_value=0)

# Renombrar las columnas y restablecer los índices
resultados.columns = ['Varones', 'Mujeres']
resultados = resultados.reset_index()

resultados['Total Fila'] = resultados['Varones'] + resultados['Mujeres']

resultados['Combinación'] = resultados.apply(lambda row: f"PP07H == {row['PP07H']} y PP07I == {row['PP07I']}", axis=1)

resultados.loc['Total Col'] = resultados.sum(numeric_only=True, axis=0)

resultados.loc[:, '% Varones'] = resultados['Varones']/resultados.loc['Total Col', 'Varones'] * 100
resultados.loc[:, '% Mujeres'] = resultados['Mujeres']/resultados.loc['Total Col', 'Mujeres'] * 100

resultados = resultados[['Combinación', 'Varones', '% Varones', 'Mujeres', '% Mujeres', 'Total Fila']]

# Aplicar formato a los valores en la tabla
tabla_pob_ocupada_asalariada = resultados.style.format({
    'Varones': '{:,.0f}',
    '% Varones': '{:,.2f}',
    'Mujeres': '{:,.0f}',
    '% Mujeres': '{:,.2f}',
    'Total Fila': '{:,.0f}'
})

tabla_pob_ocupada_asalariada

,Combinación,Varones,% Varones,Mujeres,% Mujeres,Total Fila
0,PP07H == 1.0 y PP07I == 0.0,"4,451,936",63.75,"3,397,674",60.31,"7,849,610"
1,PP07H == 2.0 y PP07I == 0.0,"27,918",0.40,"13,121",0.23,"41,039"
2,PP07H == 2.0 y PP07I == 1.0,"277,412",3.97,"252,810",4.49,"530,222"
3,PP07H == 2.0 y PP07I == 2.0,"2,209,098",31.64,"1,965,206",34.88,"4,174,304"
4,PP07H == 2.0 y PP07I == 9.0,"16,529",0.24,"4,650",0.08,"21,180"
Total Col,nan,"6,982,893",100.00,"5,633,461",100.00,"12,616,354"


In [11]:
# Componente: Inserción laboral

# T4. 
# Variable: Participación en el trabajo remunerado
# Indicador: Ratio M/V en las horas trabajadas remuneradas

def participacion_trabajo_remunerado(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):
    
    """
    Calcula el acceso a derechos laborales, a partir de la proporción en las horas trabajadas remuneradas entre hombres y mujeres junto con los errores asociados.

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tasa (DataFrame): DataFrame con las horas trabajadas remuneradas (M/V) desglosadas por Aglomerado o Provincia. Expresado en % [0-100]
        size (DataFrame): DataFrame con los numeros de observaciones utilizados para calcular las horas trabajadas remuneradas (M/V). Expresado en valores absolutos
        ratio (DataFrame): DataFrame con las proporciones de las horas trabajadas remuneradas (M/V). Expresado en % [0-100]
        error (DataFrame): DataFrame con los errores asociados a las proporciones. Expresado en % [0-100]. 
                            LI: Limite inferior, LS: Limite superior, ME: Margen de error, ER: Error relativo (CV)


    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')
    df_estado = df_temp[(df_temp['ESTADO'] == 1)]          # personas ocupadas 

    numerador = df_estado[(df_estado['PP3E_TOT']>0) & (df_estado['PP3E_TOT']!=999)].groupby(['CH04', 'PROVINCIA'])['PP3E_TOT'].sum().unstack(level=0)
    denominador = df_estado[(df_estado['PP3E_TOT']>0) & (df_estado['PP3E_TOT']!=999)].groupby(['CH04', 'PROVINCIA'])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_temp[df_temp['ESTADO'] == 2].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Participacion trabajo remunerado'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [12]:
tasa, size, ratio, error = participacion_trabajo_remunerado(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)

horas_remuneradas = pd.concat([tasa, size, ratio, error], axis=1)

horas_remuneradas

,Varon,Mujer,N_v,N_m,Participacion trabajo remunerado,LI,LS,ME,ER
PROVINCIA,,,,,,,,,
2,3.432555,2.916838,39,36,84.975709,77.046860,92.904558,7.928849,9.330724
6,4.160159,3.458459,301,342,83.132875,80.159870,86.105880,2.973005,3.576209
10,54.764366,43.321178,55,59,79.104683,60.859253,97.350113,18.245430,23.064918
14,9.053359,7.711154,129,164,85.174507,78.756684,91.592329,6.417822,7.534910
18,17.474204,14.617914,43,27,83.654251,66.149809,101.158693,17.504442,20.924749
22,13.978736,12.744291,44,27,91.169122,74.945821,107.392423,16.223301,17.794732
26,36.093737,31.033598,71,43,85.980562,68.204067,103.757056,17.776495,20.675016
30,22.276719,16.630567,94,96,74.654475,63.418928,85.890022,11.235547,15.050065
34,33.362570,29.491209,50,13,88.396095,60.373617,116.418574,28.022479,31.701037


In [13]:
df_temp = df_people_pool.query('CH06 >= 16 & CH06 < 65')
df_estado = df_temp[(df_temp['ESTADO'] == 1)]          # personas ocupadas asalariadas

# Filtrar y contar las combinaciones por género
resultados = df_estado.groupby(['CH04', 'CAT_OCUP', 'PP03C', 'PP03D'])['PONDERA_new'].sum().unstack(level='CH04', fill_value=0)

# Renombrar las columnas y restablecer los índices
resultados.columns = ['Varones', 'Mujeres']
resultados = resultados.reset_index()

resultados['Total Fila'] = resultados['Varones'] + resultados['Mujeres']

resultados['Combinación'] = resultados.apply(lambda row: f"CAT_OCUP == {row['CAT_OCUP']}, PP03C == {row['PP03C']}, PP03D == {row['PP03D']}", axis=1)

resultados.loc['Total Col'] = resultados.sum(numeric_only=True, axis=0)

resultados.loc[:, '% Varones'] = resultados['Varones']/resultados.loc['Total Col', 'Varones'] * 100
resultados.loc[:, '% Mujeres'] = resultados['Mujeres']/resultados.loc['Total Col', 'Mujeres'] * 100

resultados = resultados[['Combinación', 'Varones', '% Varones', 'Mujeres', '% Mujeres', 'Total Fila']]

# Aplicar formato a los valores en la tabla
tabla_pob_ocupada = resultados.style.format({
    'Varones': '{:,.0f}',
    '% Varones': '{:,.2f}',
    'Mujeres': '{:,.0f}',
    '% Mujeres': '{:,.2f}',
    'Total Fila': '{:,.0f}'
})

tabla_pob_ocupada

,Combinación,Varones,% Varones,Mujeres,% Mujeres,Total Fila
0,"CAT_OCUP == 1.0, PP03C == 0.0, PP03D == 0.0","9,305",0.09,"4,898",0.07,"14,203"
1,"CAT_OCUP == 1.0, PP03C == 1.0, PP03D == 0.0","407,295",4.13,"151,377",2.04,"558,672"
2,"CAT_OCUP == 1.0, PP03C == 2.0, PP03D == 2.0","27,592",0.28,"7,503",0.10,"35,095"
3,"CAT_OCUP == 1.0, PP03C == 2.0, PP03D == 3.0","2,126",0.02,"4,270",0.06,"6,395"
4,"CAT_OCUP == 1.0, PP03C == 2.0, PP03D == 5.0",47,0.00,0,0.00,47
5,"CAT_OCUP == 2.0, PP03C == 0.0, PP03D == 0.0","24,561",0.25,"16,184",0.22,"40,744"
6,"CAT_OCUP == 2.0, PP03C == 1.0, PP03D == 0.0","2,224,770",22.56,"1,391,233",18.77,"3,616,002"
7,"CAT_OCUP == 2.0, PP03C == 2.0, PP03D == 2.0","124,964",1.27,"129,225",1.74,"254,190"
8,"CAT_OCUP == 2.0, PP03C == 2.0, PP03D == 3.0","6,906",0.07,"20,252",0.27,"27,157"
9,"CAT_OCUP == 2.0, PP03C == 2.0, PP03D == 4.0",208,0.00,933,0.01,"1,141"


### Analisis dataframes

In [16]:
# cod_provincia
dict_cod_provincia = {2: "CABA",
 6: "Buenos Aires",
 10: "Catamarca",
 14: "Córdoba",
 18: "Corrientes",
 22: "Chaco",
 26: "Chubut",
 30: "Entre Ríos",
 34: "Formosa",
 38: "Jujuy",
 42: "La Pampa",
 46: "La Rioja",
 50: "Mendoza",
 54: "Misiones",
 58: "Neuquén",
 62: "Río Negro",
 66: "Salta",
 70: "San Juan",
 74: "San Luis",
 78: "Santa Cruz",
 82: "Santa Fe",
 86: "Santiago del Estero",
 90: "Tucumán",
 94: "TdF"}

In [17]:
empleo.index = empleo.index.map(dict_cod_provincia)
desempleo.index = desempleo.index.map(dict_cod_provincia)
empleo_asalariado_con_descuento.index = empleo_asalariado_con_descuento.index.map(dict_cod_provincia)
horas_remuneradas.index = horas_remuneradas.index.map(dict_cod_provincia)

In [25]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot


# Create bar traces
bar_trace1 = go.Bar(
    x=empleo.index,
    y=empleo['Actividad productiva'],
    name='T1. Actividad productiva'
)

bar_trace2 = go.Bar(
    x=desempleo.index,
    y=desempleo['Búsqueda de trabajo'],
    name='T2. Búsqueda de trabajo'
)

bar_trace3 = go.Bar(
    x=empleo_asalariado_con_descuento.index,
    y=empleo_asalariado_con_descuento['Acceso a derechos laborales'],
    name='T3. Acceso a derechos laborales'
)

bar_trace4 = go.Bar(
    x=horas_remuneradas.index,
    y=horas_remuneradas['Participacion trabajo remunerado'],
    name='T4. Participacion trabajo remunerado'
)

# Create a layout
layout = go.Layout(
    title='Insercion laboral'
)

# Create a Figure with subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=('T1', 'T2', 'T3', 'T4'))

# Add traces to the subplots

# Add traces to the subplots
fig.add_trace(bar_trace1, row=1, col=1)
fig.add_trace(bar_trace2, row=1, col=2)
fig.add_trace(bar_trace3, row=2, col=1)
fig.add_trace(bar_trace4, row=2, col=2)

fig.update_yaxes(range=[0, 150], row=1, col=1)
fig.update_yaxes(range=[0, 150], row=1, col=2)
fig.update_yaxes(range=[0, 150], row=2, col=1)
fig.update_yaxes(range=[0, 150], row=2, col=2)


# Update the layout for each subplot
fig.update_annotations(
    dict(x=0.5, y=1.15, showarrow=False, text="Plot 1"),
    selector=dict(title="T1")
)

fig.update_annotations(
    dict(x=0.5, y=1.15, showarrow=False, text="Plot 2"),
    selector=dict(title="T2")
)

fig.update_annotations(
    dict(x=0.5, y=1.15, showarrow=False, text="Plot 3"),
    selector=dict(title="T3")
)

fig.update_annotations(
    dict(x=0.5, y=1.15, showarrow=False, text="Plot 4"),
    selector=dict(title="T4")
)

file_path = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/'
filename = 'actividad_productiva.html'
fig.update_layout(layout)
plot(fig, filename=file_path + filename, auto_open=False)


'/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/figs/actividad_productiva.html'